In [ ]:
import IPython.display as ipd
import torch
import soundfile as sf
from tqdm import tqdm
from api import StableTTSAPI

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tts_model_path = 'StableTTS-sfm-vctk.pt' # path to StableTTS checkpoint

vocoder_model_path = 'vocoders/pretrained/vocos.pt'
vocoder_type = 'vocos'

model = StableTTSAPI(tts_model_path, vocoder_model_path, vocoder_type)
model.to(device)

tts_param, vocoder_param = model.get_params()
print(f'tts_param: {tts_param}, vocoder_param: {vocoder_param}')

In [ ]:
language = 'english'
solver = 'dopri5' # recommend using euler, midpoint or dopri5
steps = 30 # the size of output trajectory, meaningless for the steps of adaptive-step solvers
cfg = 3 # recommend 1-4
temperature = 1.0
alpha = 3.0
length_scale = 1.0

# folder = "xxx"
# import os
# if os.path.exists(folder):
#     os.system(f"rm -r {folder}")
# os.makedirs(folder, exist_ok=True)

with open("./data/VCTK/vctk_audio_sid_text_test_filelist.txt", "r") as f:
    test_data = f.readlines()

SEED = 1234
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

tps = 0. 
sigma_ps = 0.
for d in tqdm(test_data):
    filename = d.split("|")[0].split("/")[-1]
    text = d.split("|")[-1].strip("\n")
    spk = int(d.split("|")[1])
    audio_output, outputs, tp, sigma_p = model.inference(text, spk, language, steps, temperature, alpha, length_scale, solver, cfg)
    break
    # tps += tp 
    # sigma_ps += sigma_p
    # sf.write(f'{folder}/{filename}', audio_output[0], model.mel_config.sample_rate, 'PCM_24')

# tps = round(tps/len(test_data), 8)
# sigma_ps = round(sigma_ps/len(test_data), 8)
# print(tps, sigma_ps)

ipd.display(ipd.Audio(audio_output[0], rate=model.mel_config.sample_rate))